In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import webbrowser
import os

In [ ]:
sia=SentimentIntensityAnalyzer()

In [ ]:
apps_df= pd.read_csv('PlayStoreData.csv')
reviews_df=pd.read_csv('UserReviews.csv')

In [ ]:
apps_df['Reviews'] = pd.to_numeric(apps_df['Reviews'], errors='coerce')

In [ ]:
apps_df.dtypes

In [ ]:
reviews_df['sentiment_score'] = reviews_df['Translated_Review'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

In [ ]:
reviews_df.head()

In [ ]:
bins = [0, 2, 4, 5] 
labels = ['1-2 stars', '3-4 stars', '4-5 stars']  # Label for each bin
apps_df.loc[:, 'rating_group'] = pd.cut(apps_df['Rating'], bins=bins, labels=labels, right=False)

In [ ]:
apps_df.head()

In [ ]:
merged_df = pd.merge(apps_df, reviews_df, on="App")
merged_df = merged_df.drop_duplicates()

In [ ]:
merged_df.head()

In [ ]:
filtered_df=merged_df[merged_df['Reviews']>1000]

In [ ]:
print(filtered_df['Category'].value_counts())

In [ ]:
top_categories = ['GAME', 'FAMILY', 'HEALTH_AND_FITNESS', 'TRAVEL_AND_LOCAL', 'TOOLS']

In [ ]:
top_Category_df = filtered_df[filtered_df['Category'].isin(top_categories)]

In [ ]:
print(top_Category_df[['App', 'Category', 'Reviews', 'Rating','sentiment_score','rating_group']])  

In [ ]:
import plotly.express as px

In [ ]:
top_Category_df.loc[:, 'Sentiment_Category'] = top_Category_df['sentiment_score'].apply(
    lambda x: 'Positive' if x > 0.1 else ('Negative' if x < -0.1 else 'Neutral')
)

In [ ]:
sentiment_counts = top_Category_df.groupby(['rating_group', 'Sentiment_Category'], observed=True).size().reset_index(name='count')

In [ ]:
import plotly.express as px
import plotly.io as pio

In [ ]:
from IPython.display import display

In [ ]:
html_files_path="./"
if not os.path.exists(html_files_path):
    os.makedirs(html_files_path)

In [ ]:
plot_containers=""

In [ ]:
# Save each Plotly figure to an HTML file
def save_plot_as_html(fig, filename, insight):
    global plot_containers
    filepath = os.path.join(html_files_path, filename)
    html_content = pio.to_html(fig, full_html=False, include_plotlyjs='inline')
    # Append the plot and its insight to plot_containers
    plot_containers += f"""
    <div class="plot-container" id="{filename}" onclick="openPlot('{filename}')">
        <div class="plot">{html_content}</div>
        <div class="insights">{insight}</div>
    </div>
    """
    fig.write_html(filepath, full_html=False, include_plotlyjs='inline')

In [ ]:
plot_width=600
plot_height=300
plot_bg_color='black'
text_color='white'
title_font={'size':16}
axis_font={'size':12}

In [ ]:
fig1= px.bar(
    sentiment_counts, 
    x='rating_group',  # x-axis: Rating group (e.g., 1-2 stars, 3-4 stars, etc.)
    y='count',  # y-axis: Sentiment count
    color='Sentiment_Category',  # Color by Sentiment_Category (Positive, Negative, Neutral)
    title='Sentiment Distribution by Rating Group',  # Chart title
    labels={'rating_group': 'Rating Group', 'count': 'Sentiment Count', 'Sentiment_Category': 'Sentiment'},  # Labels for axes
    barmode='stack',  # Stack the bars
    width=800,  # Adjust width of the chart
    height=500  # Adjust height of the chart
)
fig1.update_layout(
        xaxis_title='Category',
        yaxis_title='Rating',
        plot_bgcolor='black',
        paper_bgcolor='black',
        font_color='white',
        title_font=dict(size=16),
        xaxis=dict(title_font=dict(size=12)),
        yaxis=dict(title_font=dict(size=12)),
        margin=dict(l=20, r=20, t=40, b=20)
)
save_plot_as_html(fig1, "stacked bar chart.html", "This is a scattered plot")

In [ ]:
fig1.show()

In [ ]:
dashboard_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Analytics Dashboard</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #333;
            color: #fff;
            margin: 0;
            padding: 0;
        }}
        .header {{
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 20px;
            background-color: #444;
        }}
        .header img {{
            margin: 0 10px;
            height: 50px;
        }}
        .container {{
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
            padding: 20px;
        }}
        .plot-container {{
            border: 2px solid #555;
            margin: 10px;
            padding: 10px;
            width: {plot_width}px;
            height: {plot_height}px;
            overflow: hidden;
            position: relative;
            cursor: pointer;
        }}
        .insights {{
            display: none;
            position: absolute;
            right: 10px;
            top: 10px;
            background-color: rgba(0, 0, 0, 0.7);
            padding: 5px;
            border-radius: 5px;
            color: #fff;
        }}
        .plot-container:hover .insights {{
            display: block;
        }}
    </style>
    <script>
        function openPlot(filename) {{
            window.open(filename, '_blank');
        }}
    </script>
</head>
<body>
    <div class="header">
        <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Logo_2013_Google.png/800px-Logo_2013_Google.png" alt="Google Logo">
        <h1>Analytics Dashboard</h1>
        <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/Google_Play_Store_badge_EN.svg/1024px-Google_Play_Store_badge_EN.svg.png" alt="Google Play Store Logo">
    </div>
    <div class="container">
        {plots}
    </div>
</body>
</html>
"""

In [ ]:
import webbrowser
import os

In [ ]:
final_html = dashboard_html.format(plots=plot_containers, plot_width=plot_width, plot_height=plot_height)

In [ ]:
dashboard_path=os.path.join(html_files_path,'web page.html')

In [ ]:
with open(dashboard_path, "w", encoding="utf-8") as f:
    f.write(final_html)

In [ ]:
webbrowser.open('file://' + os.path.realpath(dashboard_path))